# 示例代码

In [49]:
from sklearn.naive_bayes import MultinomialNB

# 数据生成
X_train_dict = ["名作", "电影", "华丽", "动作", "世界", "感动", "沙尘暴", "火星", "探测", "重新开始", "VR"]
X_train_data = [["那部让人感动的电影名作重映"],
                ["华丽的动作电影首映"],
                ["复映的名作感动了世界"],
                ["沙尘暴笼罩着火星"],
                ["火星探测终于重新开始"],
                ["VR中看到的火星沙尘暴让人感动"]]
y_train_data = [ "宇宙","电影"]

X_train = [ [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
            [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
            [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1]]
y_train = [1, 1, 1, 0, 0, 0]
X_test_data=["复映的动作电影名作让人感动"]
X_test = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]]

print(X_train_dict)
for index in range(len(X_train)):
    print(X_train[index], y_train[index])
    print(X_train_data[index],y_train_data[y_train[index]])

model = MultinomialNB()
model.fit(X_train, y_train) # 训练
type=model.predict(X_test) # 评估

print("output:",X_test_data, y_train_data[type[0]])

#print(X_train_data[y_pred])

['名作', '电影', '华丽', '动作', '世界', '感动', '沙尘暴', '火星', '探测', '重新开始', 'VR']
[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0] 1
['那部让人感动的电影名作重映'] 电影
[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0] 1
['华丽的动作电影首映'] 电影
[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0] 1
['复映的名作感动了世界'] 电影
[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0] 0
['沙尘暴笼罩着火星'] 宇宙
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0] 0
['火星探测终于重新开始'] 宇宙
[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1] 0
['VR中看到的火星沙尘暴让人感动'] 宇宙
output: ['复映的动作电影名作让人感动'] 电影


回归 GaussianNB
分类 MultinomialNB

In [ ]:
import numpy as np

class MultinomialNB(object):
    def __init__(self,alpha=1.0,fit_prior=True,class_prior=None):
        self.alpha = alpha
        self.fit_prior = fit_prior
        self.class_prior = class_prior
        self.classes = None
        self.conditional_prob = None

    def _calculate_feature_prob(self,feature):
        values = np.unique(feature)
        total_num = float(len(feature))
        value_prob = {}
        for v in values:
            value_prob[v] = np.sum(np.equal(feature,v)+self.alpha)/(total_num+len(values)*self.alpha)
        return value_prob

    def fit(self,X,y):
        self.classes = np.unique(y)
        if self.class_prior == None:
            class_num = len(self.classes)
            if not self.fit_prior:
                self.class_prior = [1.0/class_num for _ in range(class_num)]
            else:
                self.class_prior = []
                sample_num = float(len(y))
                for c in self.classes:
                    c_num = np.sum(np.equal(y,c))
                    self.class_prior.append((c_num+self.alpha)/(sample_num+class_num*self.alpha))
        self.conditional_prob = {}
        for c in self.classes:
            self.conditional_prob[c] = {}
            for i in range(len(X[0])):
                feature = X[np.equal(y,c)][:,i]
                self.conditional_prob[c][i] = self._calculate_feature_prob(feature)
        return self

    def _get_xj_prob(self,values_prob,target_value):
        return values_prob[target_value]

    def _predict_single_sample(self,x):
        label = -1
        max_posterior_prob = 0
        for c_index in range(len(self.classes)):
            current_class_prior = self.class_prior[c_index]

        current_conditional_prob = 1.0
        feature_prob = self.conditional_prob[self.classes[c_index]]
        j = 0
        for feature_i in feature_prob.keys():
            current_conditional_prob *= self._get_xj_prob(feature_prob[feature_i],x[j])
            j += 1

            if current_class_prior * current_conditional_prob > max_posterior_prob:
                max_posterior_prob = current_class_prior * current_conditional_prob
                label = self.classes[c_index]
        return label

    def predict(self,X):
        if X.ndim == 1:
            return self._predict_single_sample(X)
        else:
            labels = []
            for i in range(X.shape[0]):
                label = self._predict_single_sample(X[i])
                labels.append(label)
            return labels
    
class GaussianNB(MultinomialNB):     
    def _calculate_feature_prob(self,feature):         
        mu = np.mean(feature)         
        sigma = np.std(feature)         
        return (mu,sigma)

    def _prob_gaussion(self,mu,sigma,x):
        return 1.0/(sigma*np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2/(2*sigma**2))

    def _get_xj_prob(self,mu_sigma,target_value):
        return self._prob_gaussion(mu_sigma[0],mu_sigma[1],target_value)


X = np.array(    [ [1,1,1,1,1,2,2,2,2,2,3,3,3,3,3],
                   [4,5,5,4,4,4,5,5,6,6,6,5,5,6,6] ])
X = X.T     
y = np.array([-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,-1])


#nb = MultinomialNB(alpha=1.0,fit_prior=True)
nb = GaussianNB(alpha=1.0,fit_prior=True)
nb.fit(X,y)
print(nb.alpha)
print(nb.class_prior)
print(nb.classes)
print(nb.conditional_prob)
print(nb.predict(np.array([2,4])))